## Register Data with Athena

In [2]:
# import boto3
# import sagemaker

# sess = sagemaker.Session()
# bucket = sess.default_bucket()
# role = sagemaker.get_execution_role()
# region = boto3.Session().region_name


import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [3]:
ingest_create_athena_table_csv_passed = False

In [4]:
%store -r ingest_create_athena_db_passed

In [5]:
try:
    ingest_create_athena_db_passed
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not create the Athena Database.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

In [6]:
print(ingest_create_athena_db_passed)

True


In [7]:
if not ingest_create_athena_db_passed:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not create the Athena Database.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

[OK]


In [8]:
s3_income_csv = "s3://projectbucketvsbr22/income/"


In [9]:
print(ingest_create_athena_db_passed)

True


In [10]:
from pyathena import connect

In [11]:
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [12]:
database_name = "personalincome"
table_name_csv = "dsp_income"

In [13]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

# Create Schema for Personal Income 

In [14]:
# # SQL statement to execute

statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{} (
  `date` date,
  `value` int
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES ('field.delim' = ',')
LOCATION '{}'
TBLPROPERTIES ('classification' = 'csv');""".format(
    database_name, table_name_csv, s3_income_csv
)
print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS personalincome.dsp_income (
  `date` date,
  `value` int
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES ('field.delim' = ',')
LOCATION 's3://projectbucketvsbr22/income/'
TBLPROPERTIES ('classification' = 'csv');


In [15]:
import pandas as pd

pd.read_sql(statement, conn)

""


In [16]:
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(10)

,tab_name
0,dsp_income


In [17]:
if table_name_csv in df_show.values:
    ingest_create_athena_table_csv_passed = True

In [18]:
%store ingest_create_athena_table_csv_passed

Stored 'ingest_create_athena_table_csv_passed' (bool)


In [19]:
statement = """SELECT * FROM {}.{}
    LIMIT 10""".format(
    database_name, table_name_csv
)

print(statement)

SELECT * FROM personalincome.dsp_income
    LIMIT 10


In [20]:
df = pd.read_sql(statement, conn)
df.head(5)

,date,value
0,None,None
1,None,None
2,None,None
3,None,None
4,None,None


## Release Resources

In [21]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [22]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>